In [7]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [8]:
url = "https://redplanetscience.com/"

In [12]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
    
browser.visit(url)
    
html = browser.html
soup = BeautifulSoup(html, "html.parser")

headlines = soup.findAll("div", class_="content_title")
articles = soup.findAll("div", class_="article_teaser_body")
    
# Quit the browser
browser.quit()



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\Wayssen\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [13]:
# Extracting titles and articles
titles = []
paragraphs = []

for headline in headlines:
    titles.append(headline.get_text())
    
for article in articles:
    paragraphs.append(article.get_text())

In [16]:
titles

["10.9 Million Names Now Aboard NASA's Perseverance Mars Rover",
 "Mars Helicopter Attached to NASA's Perseverance Rover",
 'Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth',
 "Common Questions about InSight's 'Mole'",
 "NASA-JPL Names 'Rolling Stones Rock' on Mars",
 "Q&A with the Student Who Named Ingenuity, NASA's Mars Helicopter",
 "NASA's Mars Perseverance Rover Passes Flight Readiness Review",
 "NASA InSight's 'Mole' Is Out of Sight",
 "NASA Engineers Checking InSight's Weather Sensors",
 'NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network',
 "NASA's Perseverance Rover Will Carry First Spacesuit Materials to Mars",
 '7 Things to Know About the Mars 2020 Perseverance Rover Mission',
 'Two Rovers to Roll on Mars Again: Curiosity and Mars 2020',
 "NASA's InSight Flexes Its Arm While Its 'Mole' Hits Pause",
 "NASA's Perseverance Rover 100 Days Out"]